In [ ]:
from gurobipy import Model, GRB, quicksum

# Paramètres (exemple simplifié)
d_trains = {"T1": 480, "T2": 510}  # Arrivée des trains en minutes depuis minuit (ex: 08:00 -> 480 min)
d_durations = {"DEB": 30, "FORM": 40, "DEG": 20}  # Durées des tâches (en min)
d_machines = {"DEB": "M1", "FORM": "M2", "DEG": "M3"}  # Association machine-tâche
d_machine_unavailability = {"M2": [(780, 1560)]}  # Indispo M2 samedi 13h- lundi 13h
d_min_gap = {"ARR_TO_DEB": 60, "DEB_TO_FORM": 0, "FORM_TO_DEG": 0, "DEG_TO_DEP": 0}

# Création du modèle
m = Model("Optimisation_Triage")



In [ ]:
# Variables de décision : heures de début des tâches machines
s = {(t, task): m.addVar(vtype=GRB.INTEGER, name=f"s_{t}_{task}") for t in d_trains for task in d_durations}

# Contraintes : respect des précédences
for t in d_trains:
    m.addConstr(s[t, "DEB"] >= d_trains[t] + d_min_gap["ARR_TO_DEB"])  # Débranchement après arrivée
    m.addConstr(s[t, "FORM"] >= s[t, "DEB"] + d_durations["DEB"] + d_min_gap["DEB_TO_FORM"])  # Formation après débranchement
    m.addConstr(s[t, "DEG"] >= s[t, "FORM"] + d_durations["FORM"] + d_min_gap["FORM_TO_DEG"])  # Dégarage après formation

# Contraintes : indisponibilités des machines
for t in d_trains:
    for task, machine in d_machines.items():
        if machine in d_machine_unavailability:
            for (start, end) in d_machine_unavailability[machine]:
                m.addConstr((s[t, task].X < start) | (s[t, task].X > end))  # Pas pendant l'indispo

# Objectif : minimiser l'étalement des tâches
m.setObjective(quicksum(s[t, "DEG"] for t in d_trains), GRB.MINIMIZE)

# Résolution
m.optimize()

# Affichage des résultats
if m.status == GRB.OPTIMAL:
    for t in d_trains:
        for task in d_durations:
            print(f"Train {t}, Tâche {task} : {int(s[t, task].x)} min")

TypeError: '<' not supported between instances of 'Var' and 'int'